# Predicting Credit Card Default with Neural Networks

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline

### Back with the credit card default dataset

In [2]:
# Loading the dataset
DATA_DIR = '../data'
FILE_NAME = 'credit_card_default.csv'
data_path = os.path.join(DATA_DIR, FILE_NAME)
ccd = pd.read_csv(data_path, index_col="ID")
ccd.rename(columns=lambda x: x.lower(), inplace=True)
ccd.rename(columns={'default payment next month':'default'}, inplace=True)

# getting the groups of features
bill_amt_features = ['bill_amt'+ str(i) for i in range(1,7)]
pay_amt_features = ['pay_amt'+ str(i) for i in range(1,7)]
numerical_features = ['limit_bal','age'] + bill_amt_features + pay_amt_features

# Creating creating binary features
ccd['male'] = (ccd['sex'] == 1).astype('int')
ccd['grad_school'] = (ccd['education'] == 1).astype('int')
ccd['university'] = (ccd['education'] == 2).astype('int')
#ccd['high_school'] = (ccd['education'] == 3).astype('int')
ccd['married'] = (ccd['marriage'] == 1).astype('int')

# simplifying pay features 
pay_features= ['pay_' + str(i) for i in range(1,7)]
for x in pay_features:
    ccd.loc[ccd[x] <= 0, x] = 0

# simplifying delayed features
delayed_features = ['delayed_' + str(i) for i in range(1,7)]
for pay, delayed in zip(pay_features, delayed_features):
    ccd[delayed] = (ccd[pay] > 0).astype(int)
    
# creating a new feature: months delayed
ccd['months_delayed'] = ccd[delayed_features].sum(axis=1)

## Split and standarize the dataset

In [3]:
numerical_features = numerical_features + ['months_delayed']
binary_features = ['male','married','grad_school','university']
X = ccd[numerical_features + binary_features]
y = ccd['default'].astype(int)

## Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=5/30, random_state=101)

## Standarize
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train[numerical_features])
X_train.loc[:, numerical_features] = scaler.transform(X_train[numerical_features])
# Standarize also the testing set
X_test.loc[:, numerical_features] = scaler.transform(X_test[numerical_features])

C:\Users\direc\Anaconda3\envs\mybook\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


### Building the neural network for classification

In [4]:
from keras.models import Sequential
nn_classifier = Sequential()

C:\Users\direc\Anaconda3\envs\mybook\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
from keras.layers import Dense
n_input = X_train.shape[1]
n_units_hidden = 64
nn_classifier.add(Dense(units=n_units_hidden, activation='relu', input_shape=(n_input,)))

In [6]:
# add 2nd hidden layer
nn_classifier.add(Dense(units=n_units_hidden, activation='relu'))
# add 3th hidden layer
nn_classifier.add(Dense(units=n_units_hidden, activation='relu'))
# add 4th hidden layer
nn_classifier.add(Dense(units=n_units_hidden, activation='relu'))
# add 5th hidden layer
nn_classifier.add(Dense(units=n_units_hidden, activation='relu'))

In [7]:
# output layer
nn_classifier.add(Dense(1, activation='sigmoid'))

### Training the network

In [8]:
## compiling step
nn_classifier.compile(loss='binary_crossentropy', optimizer='adam')

In [9]:
nn_classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                1280      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 65        
Total params: 17,985
Trainable params: 17,985
Non-trainable params: 0
_________________________________________________________________


In [10]:
nn_classifier.save_weights('class_initial_w.h5')

In [11]:
batch_size = 64
n_epochs = 150
nn_classifier.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size)

Epoch 1/150
25000/25000 [==============================] - 1s 30us/step - loss: 0.4635
Epoch 2/150
25000/25000 [==============================] - 0s 17us/step - loss: 0.4467
Epoch 3/150
25000/25000 [==============================] - 0s 15us/step - loss: 0.4432
Epoch 4/150
25000/25000 [==============================] - 0s 16us/step - loss: 0.4414
Epoch 5/150
25000/25000 [==============================] - 0s 15us/step - loss: 0.4399
Epoch 6/150
25000/25000 [==============================] - 0s 15us/step - loss: 0.4389
Epoch 7/150
25000/25000 [==============================] - 0s 15us/step - loss: 0.4373
Epoch 8/150
25000/25000 [==============================] - 0s 15us/step - loss: 0.4370
Epoch 9/150
25000/25000 [==============================] - 0s 15us/step - loss: 0.4344
Epoch 10/150
25000/25000 [==============================] - 0s 15us/step - loss: 0.4332
Epoch 11/150
25000/25000 [==============================] - 0s 15us/step - loss: 0.4330
Epoch 12/150
25000/25000 [===============

## Evaluating predictions

In [12]:
## Getting the probabilities
y_pred_train_prob = nn_classifier.predict(X_train)
y_pred_test_prob = nn_classifier.predict(X_test)

## Classifications from predictions
y_pred_train = (y_pred_train_prob > 0.5).astype(int)
y_pred_test = (y_pred_test_prob > 0.5).astype(int)

In [13]:
from sklearn.metrics import accuracy_score
train_acc = accuracy_score(y_true=y_train, y_pred=y_pred_train)
test_acc = accuracy_score(y_true=y_test, y_pred=y_pred_test)
print("Train Accuracy: {:0.3f} \nTest Accuracy: {:0.3f}".format(train_acc, test_acc))

Train Accuracy: 0.923 
Test Accuracy: 0.740


## Re-training the network with less epochs

In [14]:
## load the initial weights
nn_classifier.load_weights('class_initial_w.h5')

In [15]:
batch_size = 64
n_epochs = 50
nn_classifier.compile(loss='binary_crossentropy', optimizer='adam')
nn_classifier.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size)

Epoch 1/50
25000/25000 [==============================] - 1s 32us/step - loss: 0.4642
Epoch 2/50
25000/25000 [==============================] - 0s 16us/step - loss: 0.4468
Epoch 3/50
25000/25000 [==============================] - 0s 16us/step - loss: 0.4441
Epoch 4/50
25000/25000 [==============================] - 0s 16us/step - loss: 0.4416
Epoch 5/50
25000/25000 [==============================] - 0s 16us/step - loss: 0.4396
Epoch 6/50
25000/25000 [==============================] - 0s 16us/step - loss: 0.4391
Epoch 7/50
25000/25000 [==============================] - 0s 16us/step - loss: 0.4377
Epoch 8/50
25000/25000 [==============================] - 0s 16us/step - loss: 0.4363
Epoch 9/50
25000/25000 [==============================] - 0s 16us/step - loss: 0.4352
Epoch 10/50
25000/25000 [==============================] - 0s 16us/step - loss: 0.4337
Epoch 11/50
25000/25000 [==============================] - 0s 16us/step - loss: 0.4322
Epoch 12/50
25000/25000 [===========================

In [16]:
## Getting the probabilities
y_pred_train_prob = nn_classifier.predict(X_train)
y_pred_test_prob = nn_classifier.predict(X_test)

## Classifications from predictions
y_pred_train = (y_pred_train_prob > 0.5).astype(int)
y_pred_test = (y_pred_test_prob > 0.5).astype(int)

## Calculating accuracy
train_acc = accuracy_score(y_true=y_train, y_pred=y_pred_train)
test_acc = accuracy_score(y_true=y_test, y_pred=y_pred_test)
print("Train Accuracy: {:0.3f} \nTest Accuracy: {:0.3f}".format(train_acc, test_acc))

Train Accuracy: 0.852 
Test Accuracy: 0.786
